In [220]:
class NeuralNetwork(object):
    """  
    Hold all layers together and propogate values through the net.
    """
    def __init__(self, data):
        """
        Hold data ie weights thresholds and input.
        also holds neurons for each layer
        """
        # create layers
        self.input_layer = data["num_input_nodes"]
        self.hidden_layer = data["num_hidden_nodes"]
        self.output_layer = data["num_output_nodes"]
        # inputs
        self.inputs = data["inputs"]
        # weights
        self.first_weights = data["first_weights"]
        self.second_weights = data["second_weights"]
        self.third_weights = data["third_weights"]
        # thresholds
        self.first_thresholds = data["first_thresholds"]
        self.second_thresholds = data["second_thresholds"]
        self.third_thresholds = data["third_thresholds"]
    
    def __neural_output(self, weights, inputs, threshold):
        """
        in: weights, inputs, threshold
        out: 1 or 0
        
        if the (sum of weights) x (sum of inputs)
        is greater than my threshold
        output 1
        else 0
        """
        length = len(inputs)
        activation = 0
        output = 0
        for i in range(length):
            activation += weights[i] * inputs[i]
        if activation > threshold:
            output = 1
        return output
    
    def __layer_out(self, weights, inputs, thresholds):
        """  
        for all input & weights, compute each output from each neuron.
        """
        outs = []
        # number of thresholds = number of output bits
        for threshold in thresholds:
            outs.append(
                self.__neural_output(weights,
                              inputs,  
                              threshold))
        return outs
    
    def one_out(self):
        """
        return bits for input layer.
        """
        return self.__layer_out(self.first_weights, 
                              self.inputs, 
                              self.first_thresholds)
    
    def hidden_layer_out(self):
        """
        return bits for hidden layer.
        """
        return self.__layer_out(self.second_weights, 
                              self.one_out(), 
                              self.second_thresholds)
    
    def output_layer_out(self):
        """
        return bits for ouput layer.
        """
        return self.__layer_out(self.third_weights, 
                              self.hidden_layer_out(), 
                              self.third_thresholds)
    
    def out(self):
        """
        for show
        """
        return self.output_layer_out()

In [4]:
def nn_info(inp,inl,hid,out):
    """
    given hidden input and out
    gives number of weights0 + weights1 + weights2 + thresholds needed
    """
    w0 = inp * inl
    w1 = inl * hid
    w2 = hid * out
    p = lambda x, y, z: "Weights from {0} to {1}: {2}\n".format(x, y, z)
    print p("input", "layer one", str(w0))
    print p("layer one", "layer two", str(w1))
    print p("layer two", "layer three", str(w2))
    print "total weights: {0}\n".format(w0+w1+w2)
    print "{0} thresholds are needed. \n".format(inp+hid+out)
    print "total: ",w0 + w1 + w2 + inp + inl + hid + out, "\n"
    return w0, w1, w2, inl,hid,out

print nn_info(69, 9, 5, 20)

In [222]:
planets = [[200,100,50,50,1],[400,100,50,50,1],[600,100,50,50,1]]

def to_bin(data, length):
    return bin(int(data))[2:].zfill(length)

def planet_to_bin_rep(planets):
#     print planets
    planet_one = [to_bin(planets[0][0], 10),
                  to_bin(planets[0][1], 10),
                  to_bin(planets[0][4], 3)]
    planet_two = [to_bin(planets[1][0], 10),
                  to_bin(planets[1][1], 10),
                  to_bin(planets[1][4], 3)]
    planet_three = [to_bin(planets[2][0], 10),
                    to_bin(planets[2][1], 10),
                    to_bin(planets[2][4], 3)]
    return planet_one, planet_two, planet_three

def full_input(l):
    value = ""
    for i in l:
        for d in i:
            value += d
    return [int(i) for i in value]

# Genetic Algorithm

### in: none
### out: weights, thresholds



In [4]:
import random as r

def init_population(pop_size):
    population = []
    for _ in xrange(pop_size):
        chrome = []
        # size of chrome
        # total number of weights and thresholds
        for i in range(869):
            chrome.append(r.randrange(-255, 255))
        population.append(chrome)
    return population

def decode(chrome):
    """
    Weights from input to layer one: 621

    Weights from layer one to layer two: 45

    Weights from layer two to layer three: 100

    total weights: 766

    94 thresholds are needed. 

    total:  869 

    (621, 45, 100, 9, 5, 20)
    """
    synapsus0 = chrome[:621]
    synapsus1 = chrome[621:621+45]
    synapsus2 = chrome[621+45:621+45+100]
    thresholds0 = chrome[621+45+100:621+45+100+9]
    thresholds1 = chrome[621+45+100+9:621+45+100+9+5]
    thresholds2 = chrome[621+45+100+9+5:621+45+100+9+5+20]
    return synapsus0, synapsus1, synapsus2, thresholds0, thresholds1, thresholds2

def nn_data(chrome):
    """
    put the data into a dict to easily create a nn object.
    """
    decoded_chrome = decode(chrome)
    data = dict()
    data["num_input_nodes"] = 9
    data["num_hidden_nodes"] = 5
    data["num_output_nodes"] = 20
    data["inputs"] = fill_input(planet_to_bin_rep(planets))
    data["first_weights"] = decoded_chrome[0]
    data["second_weights"] = decoded_chrome[1]
    data["third_weights"] = decoded_chrome[2]
    data["first_thresholds"] = decoded_chrome[3]
    data["second_thresholds"] = decoded_chrome[4]
    data["third_thresholds"] = decoded_chrome[5]
    return data

def decode_output(out):
    """
    in: binary string
    out: x, y corrdinates
    """
    x = out[:10]
    y = out[10:]
    return int(x, 2), int(y, 2)
    
def gen():
    """
    for each chrome do

        decode chrome

        create new nn

        run nn
        
        decode output
        
        compare output to desired output

        assign fitness

        select part

        cross

        mutate
    
    create new pop
    """
    gens = 1
    pop_size = 1
    pop = init_pop(pop_size)
    for _ in xrange(gens):
        temp_pop = []
        for chrome in pop:
            data_for_network = nn_data(chrome)
            ann = NeuralNetwork(data_for_network)
            x, y = decode_output(ann.out())
            
    

621
45
100
9
5
20
([111, -246, -138, 107, 191, 123, 70, 63, -115, 16, 27, 136, 206, -199, 118, 33, 16, -244, -249, 181, -49, -149, 209, -146, 4, 114, 160, -40, 19, 12, 132, 239, 6, -110, -206, 241, 145, 159, 253, -162, 76, -20, 72, 68, -235, 116, -103, -180, 110, -240, 105, 83, 28, 135, -49, -208, -232, 89, -100, -106, 153, 81, 38, 144, -101, 218, 39, -103, -34, 51, -57, 98, 229, 75, 92, 176, -235, -114, -249, -87, -185, 54, 104, -98, 182, -136, -240, 2, -208, -247, 250, 47, 29, 182, 22, -103, 141, 183, 172, 14, -65, -241, -248, -129, 77, -57, 157, -143, -97, 250, -191, 48, 126, -138, -67, -52, 83, -249, 239, -150, -187, 238, -169, 144, -149, 115, 169, -62, -143, 195, 15, -46, 251, -24, 91, 80, 79, 170, 99, 114, 223, -250, 98, 87, -238, -65, 190, -223, 62, -145, -87, -251, 126, 79, 126, 33, -18, -182, -239, 169, 243, 121, 143, 163, -53, -59, -41, 237, -91, -140, -21, -51, -107, -193, -243, -196, 79, -75, 153, -247, -135, 81, 219, 249, -48, -47, -95, 192, -253, -29, 174, 64, 31, 193, 14

In [221]:
import random

data = {
    "num_input_nodes": 9,
    "num_hidden_nodes": 5,
    "num_output_nodes": 20,
    "inputs": full_input(planet_to_bin_rep(planets))} # y x m
    # len of from * len of to
data["first_weights"] = []# 69 * 9
data["second_weights"] = [] # 9 * 5
data["third_weights"] = [] # 5 * 20
    # len of layer
data["first_thresholds"] = []
data["second_thresholds"] = []  
data["third_thresholds"] = []

nn = NeuralNetwork(data)
print nn.out()

In [8]:
def decode_output(out):
    """
    in: binary string
    out: x, y corrdinates
    """
    x = out[:10]
    y = out[10:]
    return int(x, 2), int(y, 2)

print decode_output("10101010010101110100")

(681, 372)
